<a href="https://colab.research.google.com/github/Taeseong-eom/ApplePython/blob/main/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/Colab Notebooks/"
!kaggle datasets download -d jessicali9530/celeba-dataset
!unzip -q celeba-dataset.zip -d .

100% 1.33G/1.33G [01:01<00:00, 24.8MB/s]
100% 1.33G/1.33G [01:01<00:00, 23.2MB/s]


In [3]:
from PIL import Image
import os
import numpy as np

이미지경로 = os.listdir('/content/img_align_celeba/img_align_celeba')

images = []

# 이미지를 다루기 편해짐
for path in 이미지경로[0:50000]: # 속도를 위해 5만개 정도만 사용.
  숫자화이미지 = Image.open('img_align_celeba/img_align_celeba/' + path).crop((20, 30, 160, 180)).convert('L').resize((64,64))# 사진 가로폭을 가운데 20px~160px , 사진 세로폭을 30px~180px 짜름.
  images.append( np.array(숫자화이미지)) # convert('L')를 사용하여 흑백으로 적용

images = np.array(images)
images = np.divide(images, 255) # 이미지를 255로 나누어 이미지를 0 ~ 1 사이로 나눠 성능 향상.

images = images.reshape(50000, 64, 64, 1)
print(images.shape)


(50000, 64, 64, 1)


In [5]:
import tensorflow as tf

discriminator = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), strides=(2,2), padding='same',input_shape=[64,64,1]), # 64개의 필터를 사용, 필터의 크기는 (3,3), 입력데이터 스캔시 이동하는 단위(stride),padding을 이용해 입력과 출력의 크기를 동일하게 조정함.
    tf.keras.layers.LeakyReLU(alpha=0.2), # 음수 입력시 alpha를 곱한 값이 출력됨. # GAN에서 좀더 좋은 성능을 보이기도 해서 사용함.
    tf.keras.layers.Dropout(0.4), # Overfitting 을 방지하기 위해 뉴런의 일부(0.4)를 비활성화 시킴.
    tf.keras.layers.Conv2D(64, (3,3), strides=(2,2), padding='same'),
    tf.keras.layers.LeakyReLU(alpha=0.2),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1,activation='sigmoid') # 0 또는 1로 출력을 제한(sigmoid)
])


In [10]:
generator = tf.keras.models.Sequential([
    tf.keras.layers.Dense(4 * 4 * 256, input_shape=(100,)), # 가로4,세로4 데이터를 256장 만들기 위한 노드수
    tf.keras.layers.Reshape((4,4,256)),# 이미지처럼 만들기 위해서 모양 바꿈.
    tf.keras.layers.Conv2DTranspose(256, 3, strides = 2, padding='same'),
    tf.keras.layers.LeakyReLU(alpha=0.2),
    tf.keras.layers.BatchNormalization(), #정규화를 통해 covariate shift 문제를 완화시킴.
    tf.keras.layers.Conv2DTranspose(128, 3, strides = 2, padding='same'),
    tf.keras.layers.LeakyReLU(alpha=0.2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2DTranspose(64, 3, strides=2, padding='same'),
    tf.keras.layers.LeakyReLU(alpha=0.2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2DTranspose(1, 3, strides=2, padding='same', activation='sigmoid')
])

In [11]:
GAN = tf.keras.models.Sequential([generator, discriminator]) # GAN 생성

discriminator.compile(optimizer='adam', loss='binary_crossentropy') # 확률을 뱉을 땐

discriminator.trainable = False # 학습된 구분모델이 잘못된 데이터를 학습하지 못하도록 막음.

GAN.compile(optimizer='adam',loss='binary_crossentropy') # GAN 을 컴파일하면 generator를 컴파일 할 수 있음